# Figures of dump 0 compared to 1D model

In [1]:
%pylab ipympl
from scipy import interpolate
from scipy import optimize
from nugridpy import astronomy as ast
from nugridpy import mesa as ms
from nugridpy import ascii_table
sys.path.insert(0,'/home/user/user/niagara_scratch_sblouin/PyPPM')
from ppmpy import ppm  
from ppmpy import ppmsetup as ps
from scipy.interpolate import interp1d
logging.getLogger("matplotlib").setLevel(logging.ERROR)

Populating the interactive namespace from numpy and matplotlib


In [2]:
def initialize(data,dir,cases,nominal_heat = 1,eos='g'):
    '''Initialize Rprof data for one run
    
    nominal_heat   nominal heat, from PPMstar code, so that X_Lfactors is factor wrt nominal heat
    '''
    for case in cases:
        data[case] = {}
        data[case]['path'] = dir+case+'/prfs' 
        data[case]['rp'] = ppm.RprofSet(data[case]['path'] )
        data[case]['rph'] = data[case]['rp'].get_history()
        data[case]['NDump'] = data[case]['rph'].get('NDump')
        data[case]['time(mins)'] = data[case]['rph'].get('time(mins)')
        data[case]['time(secs)'] = data[case]['rph'].get('time(secs)')
        data[case]['rp_one'] = data[case]['rp'].get_dump(data[case]['NDump'][0])
        data[case]['X_Lfactors'] = data[case]['rp_one'].get('totallum')/nominal_heat
        data[case]['grids'] = data[case]['rp_one'].get('Nx') 
        data[case]['eos'] = eos        
    return data

In [3]:
nominal_heat = .1485 # Nominal luminosity in code units
eos = 'r' # EOS used for the run
dir = '/home/user/user/niagara_scratch_fherwig/'
case = ['V3-1000xL-1000xK-1152-TyroneM1e4_0.1']
data = {}  
data = initialize(data,dir, case, nominal_heat = nominal_heat,eos=eos)

1127 rprof files found in '/home/user/user/niagara_scratch_fherwig/V3-1000xL-1000xK-1152-TyroneM1e4_0.1/prfs/.
Dump numbers range from 0 to 1126.


In [4]:
Rrun = data[case[0]]['rp'].get('R',0,resolution='l')
Arun = data[case[0]]['rp'].get('A',0,resolution='l')
Trun = data[case[0]]['rp'].get('T9',0,resolution='l')
Rhorun = data[case[0]]['rp'].get('Rho0',0,resolution='l')
Murun = data[case[0]]['rp'].get('Mu0',0,resolution='l')
kapparun = data[case[0]]['rp'].compute_kappa(0,boost=False)
Prun = data[case[0]]['rp'].get('P0',0,resolution='l')
beta = 8.314462*Rhorun*Trun/Murun/Prun

In [5]:
_,mass,rho,T,entropy,mu,kappa,eps,radius,pressure = genfromtxt('tyrone/0.1Msun__yr_1e4Msun_more.dat', unpack=True, skip_header=1)

In [6]:
matplotlib.rcParams.update({'font.size': 11})

In [22]:
fig, axs = subplots(4, 1, figsize=(5,12))
axs[0].plot(radius/1e8,T/1e6,label='1D model')
axs[0].plot(Rrun,Trun*1e3,'--',label='3D base state')
axs[0].set_xlim(0,15000)
axs[0].set_ylim(50,160)
axs[0].set_ylabel(r'$T$ ($10^6\,$K)')
axs[0].tick_params(axis='both',direction='in',which='both')
axs[0].yaxis.set_ticks_position('both')
axs[0].legend(frameon=False)
mass_axis,idx = unique(mass, return_index=True)
radius_axis = radius[idx]/1e8
rtom = interp1d(radius_axis, mass_axis, fill_value='extrapolate')
mtor = interp1d(mass_axis, radius_axis, fill_value='extrapolate')
new_tick_locations = mtor(array([0,200,1000,3000,5000]))
def tick_function(X):
    V = rtom(X)
    return [str(int(round(x))) for x in V]
ax2 = axs[0].twiny()       
ax2.set_xlim(axs[0].get_xlim())
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(new_tick_locations))
ax2.set_xlabel('$M_r$ ($M_\odot$)', labelpad=6)
ax2.tick_params(direction='in')

axs[1].plot(radius/1e8,pressure/1e18)
axs[1].plot(Rrun,Prun*1e19/1e18,'--')
axs[1].set_xlim(0,15000)
axs[1].set_ylim(0,1.5)
axs[1].set_ylabel(r'$P$ ($10^{18}\,$erg$\,$cm$^{-3}$)')
axs[1].tick_params(axis='both',direction='in',which='both')
axs[1].yaxis.set_ticks_position('both')
ax2 = axs[1].twinx()
ax2.plot(Rrun, beta, ':k',label=r'$\beta \equiv P_{\rm gas}/P$')
ax2.legend(frameon=False)
ax2 = axs[1].twiny()       
ax2.set_xlim(axs[0].get_xlim())
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels([])
ax2.tick_params(direction='in')

axs[2].plot(radius/1e8,mu)
axs[2].plot(Rrun,Murun,'--')
axs[2].set_xlim(0,15000)
axs[2].set_ylabel(r'$\mu$')
axs[2].tick_params(axis='both',direction='in',which='both')
axs[2].yaxis.set_ticks_position('both')
axs[2].xaxis.set_ticks_position('both')
# axs[2].set_xlabel(r'$R$ (Mm)')
axins = axs[2].inset_axes([0.14, 0.2, 0.34, 0.5])
axins.plot(radius/1e8,mu)
axins.plot(Rrun,Murun,'--')
axins.set_xlim(8000,9000)
axins.set_ylim(0.5915,0.617)
axins.tick_params(axis='both',direction='in',which='both')
axs[2].indicate_inset_zoom(axins, edgecolor="black")

axs[3].plot(radius/1e8,kappa)
axs[3].plot(Rrun,kapparun,'--')
axs[3].set_xlim(0,15000)
axs[3].set_ylim(0.24,0.32)
axs[3].set_ylabel(r'$\kappa$ (g$\,$cm$^{-2}$)')
axs[3].tick_params(axis='both',direction='in',which='both')
axs[3].yaxis.set_ticks_position('both')
axs[3].xaxis.set_ticks_position('both')
axs[3].set_xlabel(r'$R$ (Mm)')

tight_layout()
savefig('setup.pdf',bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …